In [46]:
import numpy as np
import matplotlib.pyplot as pl
import dask.array as da
from dask import compute, delayed
from dask.distributed import Client
from dask.diagnostics import ResourceProfiler
import nd2
from ddm.processing import radial_profile

In [ ]:
# client.close()

In [3]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB', processes=False)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.37.18.2:8787/status,
Dashboard: http://10.37.18.2:8787/status,Workers: 4
Total threads: 8,Total memory: 7.45 GiB
Status: running,Using processes: False
Comm: inproc://10.37.18.2/21920/1,Workers: 4
Dashboard: http://10.37.18.2:8787/status,Total threads: 8
Started: Just now,Total memory: 7.45 GiB
Comm: inproc://10.37.18.2/21920/5,Total threads: 2
Dashboard: http://10.37.18.2:54903/status,Memory: 1.86 GiB
Nanny: None,


In [47]:
# #read in data
from ddm import read_file
# # from ddm.data_handling import readLIF
# # dData = readLIF('../tests/data/testData3series.lif')

# # from ddm.data_handling import readTIF
# # dData3 = readTIF('../tests/data/21-03-31_ddm_water_control_sample.tif')

# # from ddm.data_handling import readND2
# # dData2 = readND2('../tests/data/testData10frames.nd2')


# dData = read_file('../tests/data/testData3series.lif', experiment=0)
# dData = read_file('../tests/data/testData10frames.nd2')
# dData = read_file('../data/A1_s3001.nd2')
# dData = read_file('../tests/data/21-03-31_ddm_water_control_sample.tif')

#dummy data for testing
# dData = np.random.randint(0, 256, size = (20, 512, 512))
# dData = da.random.randint(0, 256, size=(1000, 512, 512), chunks=(10, 512, 512), dtype='uint8')

In [32]:
# f = nd2.ND2File('../data/A1_s3001.nd2')
# dData = f.to_dask(wrapper=True)
# f.close()

In [48]:
# dData = dData.rechunk((20, 512, 512))
dData

Delayed('read_file-ad4078ad-f11f-4019-bd82-3081ffe575e9')

### Observations
- Rechunking a loaded nd2 file through the `nd2` package, will result in significant overhead. The array is rechunked prior to executing the any computation. The default behaviour of `nd2` is to create chunks with a single image.

In [62]:
#method 1 - high q for A = 0 - rework to optimise the FFTs???
def findMeanSqFFT(dumData):
    # sqFFT = 2*da.abs(da.fft.fft2(dumData))*np.abs(da.fft.fft2(dumData))
    sqFFT = 2*(da.abs(da.fft.fft2(dumData)))**2
    sqFFT_shift = da.fft.fftshift(sqFFT)
    sqFFTmean = da.mean(sqFFT_shift, axis = 0)
    return sqFFTmean/(np.shape(sqFFTmean)[0]*np.shape(sqFFTmean[1]))

def computeAB(sqFFTmean):
    sqFFTrad = radial_profile(sqFFTmean, (np.shape(sqFFTmean)[0]/2, np.shape(sqFFTmean)[1]/2))
    b = np.mean(sqFFTrad[-100:-50]) #change depending on size of array
    a = sqFFTrad - b
    return (a, b)

In [63]:
meanSQ = findMeanSqFFT()
# meanSQ.visualize()

In [64]:
%%time
out = meanSQ.compute()

2022-07-25 10:27:42,870 - distributed.worker - WARNING - Compute Failed
Key:       ('mul-815155e400f8111da363adfe247403dd', 0, 0, 0)
Function:  execute_task
args:      ((subgraph_callable-1ea297f1-6476-4cb6-99c8-d5bddb707702, 2, None, (<class 'tuple'>, [-2, -1]), <FramesSequenceND>
Axes: 3
Axis 'x' size: 1024
Axis 'y' size: 1024
Axis 't' size: 10
Pixel Datatype: u1, (slice(0, 10, None), slice(0, 1024, None), slice(0, 1024, None))))
kwargs:    {}
Exception: 'TypeError("\'<\' not supported between instances of \'slice\' and \'int\'")'



TypeError: '<' not supported between instances of 'slice' and 'int'

In [61]:
out

array([[4.17006476e+08, 1.25291879e+08, 1.01644121e+08, ...,
        9.32090458e+07, 1.01644121e+08, 1.25291879e+08],
       [4.98208295e+07, 8.18239356e+07, 7.80787359e+07, ...,
        5.02019325e+07, 6.54546621e+07, 1.22959409e+08],
       [1.03600254e+08, 5.31109736e+07, 7.88580821e+07, ...,
        1.29108586e+08, 6.66668549e+07, 1.51943651e+08],
       ...,
       [7.35438720e+07, 1.33815924e+08, 7.52792052e+07, ...,
        8.34068018e+07, 9.08012038e+07, 1.04746526e+08],
       [1.03600254e+08, 1.51943651e+08, 6.66668549e+07, ...,
        5.79416941e+07, 7.88580821e+07, 5.31109736e+07],
       [4.98208295e+07, 1.22959409e+08, 6.54546621e+07, ...,
        6.76490758e+07, 7.80787359e+07, 8.18239356e+07]])

In [ ]:
a = out[0][0]
b = out[0][1]


In [ ]:
pl.plot(a)

In [ ]:
# client.restart()
client.close()

In [ ]:
#method 2 - delay time is zero then A = real part of squared stuff
# fTFt = np.fft.fftshift(2*np.fft.fft2(dData2)*np.conj(np.fft.fft2(dData2, axes = (0,1))))
# aFull = np.real(np.mean(fTFt, axis = 0))
# c = radial_profile(aFull, (512, 512))
# pl.figure(0)
# #pl.plot(c[10:], color = 'r')
# pl.plot(A[10:], color = 'b')

# SqFFT = 2*np.abs(np.fft.fft2(dData2)**2)
# SqFFT = np.fft.fftshift(SqFFT)
# SqFFTmean = np.mean(SqFFT, axis = 0)
# SqFFTrad = radial_profile(SqFFTmean, (512, 512))

# d = SqFFTrad - c

# pl.figure(1)
# #pl.plot(d[10:], color = 'm')
# pl.axhline(B, color = 'c')